# Adding Blank Space in Completion Data for Petrel

In [ ]:
import pandas as pd
import numpy as np
import timeit

start = timeit.default_timer()

# read data
df = pd.read_excel(r"C:\Users\Lapi-ITB\komplesion4.xlsx",sheet_name="Sheet1")
df.info()

# drop duplicates and reset index
df = df.drop_duplicates()
df = df.reset_index(drop = True)

# adding blank row for specified index 
for i in df.index:
    if "perforation" in df['UNITS FIELD'][i+1] and "squeeze" in df['UNITS FIELD'][i+2]:
        df.loc[i+1.5] = ""
    elif "squeeze" in df['UNITS FIELD'][i+1] and "WELLNAME" in df['UNITS FIELD'][i+2]:
        df.loc[i+1.5] = ""
    elif "perforation" in df['UNITS FIELD'][i+1] and "WELLNAME" in df['UNITS FIELD'][i+2]:
        df.loc[i+1.5]= ""
    elif "WELLNAME" in df['UNITS FIELD'][i+1] and "WELLNAME" in df['UNITS FIELD'][i+2]:
        df.loc[i+1.5]= ""
    elif "plug" in df['UNITS FIELD'][i+1] and "WELLNAME" in df['UNITS FIELD'][i+2]:
        df.loc[i+1.5]= ""
    else:
        continue

    if i == 144023.0:
        break

df = df.sort_index().reset_index(drop=True)

# convert dataframe to excel file
df.to_excel("Komplesi Minus 627.xlsx")

stop = timeit.default_timer()
print('Time: ', stop - start)

# Adding Blank Space in Production and Injection Data 

In [ ]:
import pandas as pd
import timeit

start = timeit.default_timer()

# read data
df = pd.read_excel(r"D:\GG\MINAS\Production\AddProdInj.xlsx",sheet_name="Sheet1")
df.info()

# adding blank row for specified index 
for i in df.index:
    if "*NAME" in df['*FIELD'][i]:
        df.loc[i-0.5] = ""
    else:
        continue

    if i == 12107.0:
        break

df = df.sort_index().reset_index(drop=True)

# convert dataframe to excel file
df.to_excel(r"D:\GG\MINAS\Production\AddProdInj_with_Blanks.xlsx")

stop = timeit.default_timer()
print('Time: ', stop - start)

# Clearing Specific Well with Its Completion Data in Original Completion Data

In [ ]:
import pandas as pd
import openpyxl

# read the data
well_628 = pd.read_excel("D:\GG\MINAS\istMSSM1.xlsx", sheet_name="Sheet5")
df = pd.read_excel("D:\GG\MINAS\komplesi 1984.xlsx", sheet_name="Sheet1")

# drop header and NaN
df = df.drop(0, axis=0)
df = df.dropna()
df = df.reset_index(drop=True)
df

# initialization iteration variable x and y 
x = 0
y = 0

while x <= 199033:

    # stopping looping when all 628 well removed
    if y==len(well_628):
        break

    # drop the 628 well
    elif well_628["uwi"][y] in str(df["UNITS FIELD"][x]):

        # drop line that contain well name   
        df = df.drop(index = x, axis=0)
        j = x + 1

        # drop lines that contain "perforation", "squeeze", and "plug"
        while j <= 1000000:

            if "perforation" in str(df["UNITS FIELD"][j]):
                df = df.drop(index = j, axis=0)
                x += 1
            elif "squeeze" in str(df["UNITS FIELD"][j]):
                df = df.drop(index = j, axis=0)
                x += 1
            elif "plug" in str(df["UNITS FIELD"][j]):
                df = df.drop(index = j, axis=0)
                x += 1
            else: 
                break

            j += 1

        x += 1
        y = y + 1

        # print the drop step
        print(x,y-1,well_628["uwi"][y-1])
    
    # adding iterable variable (x) by 1
    else:
        
        x += 1

df.to_excel("komplesion4.xlsx")

# Filter and Visualize RFT data by Sand and Depth

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings("ignore")

# read the data
df = pd.read_excel("D:\GG\MINAS\Pressure Sumur Minas RFT.xlsx", sheet_name="Pressure HM Tiga")

# drop first four data 
df.drop([0,1,2,3], axis=0, inplace=True)
df = df.reset_index()

# create blank dataframe
dfx = pd.DataFrame()

# choose data that have equal or more than 2 data for the same layer and well
for i in df['UWI'].unique():
    for j in df['K'].unique():
        if len(df[(df['UWI']==i)&(df['K']==j)])>=2:
            dfa = df[(df['UWI']==i)&(df['K']==j)]
            dfx = pd.concat([dfx,dfa])

# dfx is the dataframe that contain 
dfx

# create visualization datum pressure vs depth for each well from the previous dataframe
for i in dfx['UWI'].unique():
    if len(dfx[dfx['UWI']==i]['K'].unique())>1:
        fig,ax=plt.subplots(figsize=(8,3))
        fig = sns.scatterplot(dfx[(df['UWI']==i)]['Datum Pressure'], dfx[(df['UWI']==i)]['TOP_DEPTH'], 
                            hue=dfx[(df['UWI']==i)]['K'], s=200)
        fig.plot(dfx[(df['UWI']==i)]['Datum Pressure'], dfx[(df['UWI']==i)]['TOP_DEPTH'], linestyle="--", color='black')
        ax.set_xlabel('Pres (psi)')
        ax.set_ylabel('Depth (ft)')
        plt.gca().invert_yaxis()
        ax.xaxis.tick_top()
        ax.xaxis.set_label_position('top')
        ax.legend()
        ax.set_title(i, fontsize=16, y=1.0, pad=-14)
        plt.grid()
        plt.show()

# Function to Visualize Pressure vs Time from RFT Data

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

# create function to visualize pressure vs time (by segment)
def visualize_segment(dfv):
    plt.style.use('seaborn-darkgrid')
    fig,ax = plt.subplots(figsize=(20,8))
    fig = sns.scatterplot(dfv['MISC_TEST_DATE_1'], dfv['Datum Pressure'], s=300)
    ax.set_xlabel('Date', fontsize = 18)
    ax.set_ylabel('Pressure (psi)', fontsize = 18)
    ax.set_title('Pressure in ' + str(dfv['SEGMMENT'].unique()[1]) + " Segment", fontsize = 25)
    plt.xticks(fontsize=20)
    plt.yticks(fontsize=20)
    ax.legend(fontsize=15, ncol=3, loc="upper left")
    plt.show()

def visualize_sand_segment(dfv):
    plt.style.use('seaborn-darkgrid')
    fig, ax = plt.subplots(figsize=(20,8))
    fig = sns.scatterplot(dfv['MISC_TEST_DATE_1'], dfv['Datum Pressure'], s=300)
    ax.set_xlabel('Date', fontsize = 18)
    ax.set_ylabel('Pressure (psi)', fontsize = 18)
    ax.set_title('Pressure in ' + str(dfv["K"].unique()[1][2:]) + "_" + str(dfv['SEGMMENT'].unique()[1]), fontsize = 25)
    plt.xticks(fontsize=20)
    plt.yticks(fontsize=20)
    ax.legend(fontsize=15, ncol=3, loc="upper left")
    plt.show()

def visualize_sand(dfv):
    plt.style.use('seaborn-darkgrid')
    fig, ax = plt.subplots(figsize=(20,8))
    fig = sns.scatterplot(dfv['TEST_DATE'], dfv['DATUM_PRESS'], s=220)
    ax.set_xlabel('Date', fontsize = 18)
    ax.set_ylabel('Pressure (psi)', fontsize = 18)
    ax.set_title('Pressure PBU in ' + str(dfv["SURVEY_SAND"].unique()), fontsize = 25)
    plt.xticks(fontsize=20)
    plt.yticks(np.arange(0, 900, 100), fontsize=20)
    ax.legend(fontsize=15, ncol=3, loc="upper left")
    plt.show()

def visualize_sand_segment2(dfv):
    plt.style.use('seaborn-darkgrid')
    fig, ax = plt.subplots(figsize=(20,8))
    fig = sns.scatterplot(dfv['TEST_DATE'], dfv['DATUM_PRESS'], s=220)
    ax.set_xlabel('Date', fontsize = 18)
    ax.set_ylabel('Pressure (psi)', fontsize = 18)
    ax.set_title('Pressure SBHP in ' + str(dfv["SURVEY_SAND"].unique()) + "_" + str(dfv["SEGMENT"].unique()), fontsize = 25)
    plt.xticks(fontsize=20)
    plt.yticks(np.arange(0, 900, 100), fontsize=20)
    ax.legend(fontsize=15, ncol=3, loc="upper left")
    plt.show()

# Find Average Pressure from RFT data and Convert It to .VOL format (by Segment)

In [ ]:
# Processing RFT data by segment

from openpyxl import load_workbook
import pandas as pd
import xlsxwriter
import timeit
import warnings
warnings.filterwarnings("ignore")

# read the data
dfz = pd.read_excel("D:\GG\MINAS\Pressure Sumur Minas RFT.xlsx", sheet_name="Pressure HM Tiga")
path = "D:\GG\Segment Rev.xlsx"
book = load_workbook(path)
writer = pd.ExcelWriter(path, engine = "openpyxl")
writer.book = book

# create data in .xlsx format for input to the .vol format converter
for i in dfz["SEGMMENT"].unique():
    if i != "ALL":
        df = pd.read_excel("D:\GG\MINAS\Pressure Sumur Minas RFT.xlsx", sheet_name = "Pressure HM Tiga")
        df = df.groupby(["SEGMMENT","UWI","MISC_TEST_DATE_1"]).mean().reset_index()
        df = df[((df["SEGMMENT"]==i)|(df["SEGMMENT"]=="ALL"))]
        visualize_segment(df)
        dfk = pd.DataFrame()
        dfk['Name'] = df['UWI']
        dfk['Day'] = df['MISC_TEST_DATE_1'].dt.day
        dfk['Month'] = df['MISC_TEST_DATE_1'].dt.month
        dfk['Year'] = df['MISC_TEST_DATE_1'].dt.year
        dfk['Pressure'] = df['Datum Pressure']
        dfk.loc[dfk['Name'].str.contains("MINA", case=False), "Name"] = str(i)
        dfk.loc[len(dfk.index)] = ["1","","","",""]
        dfk.to_excel(writer, sheet_name = str(i), index = False)
    else:
        pass

writer.save()
writer.close()

# convert excel to .vol format
start = timeit.default_timer()

wb = xlsxwriter.Workbook(r"D:\GG\Pressure by Segment.xlsx")

for segment in ["MAIN","NW"]:
    sheet1 = wb.add_worksheet(str(segment))

    row_count = 3
    df_pressure = pd.read_excel(path, sheet_name=str(segment))

    sheet1.write(0, 0, '*FIELD')
    sheet1.write(1, 0, '*DAY')
    sheet1.write(1, 1, '*MONTH')
    sheet1.write(1, 2, '*YEAR')
    sheet1.write(1, 3, '*PRESS')
    sheet1.write(2, 0, '*NAME')

    for i in df_pressure.index :
        sheet1.write(row_count, 0, int(df_pressure['Day'][i]))
        sheet1.write(row_count, 1, int(df_pressure['Month'][i]))
        sheet1.write(row_count, 2, int(df_pressure['Year'][i]))
        sheet1.write(row_count, 3, int(df_pressure['Pressure'][i]))

        if i == 1 :
            sheet1.write(2, 1, '"'+str(df_pressure['Name'][i])+'"')
        
        if str(df_pressure['Name'][i+1]) == '1' : 
            break

        if str(df_pressure['Name'][i]) != str(df_pressure['Name'][i+1]) :
            row_count+=2
            sheet1.write(row_count, 0, '*NAME')
            sheet1.write(row_count, 1, '"'+str(df_pressure['Name'][i+1])+'"')
            row_count+=1

        elif str(df_pressure['Name'][i]) == str(df_pressure['Name'][i+1]) :
            row_count+=1

wb.close()

stop = timeit.default_timer()
print('Time: ', stop - start)

# Find Max Pressure from RFT data and Convert It to .VOL format (by Sand by Segment)

In [ ]:
# Processing RFT data by sand and by segment

from openpyxl import load_workbook
import pandas as pd
import xlsxwriter
import timeit
import warnings
warnings.filterwarnings("ignore")

# read the data
dfz = pd.read_excel("D:\GG\MINAS\Pressure Sumur Minas RFT.xlsx", sheet_name="Pressure HM Tiga")
path = "D:\GG\Sand and Segment Rev.xlsx"
book = load_workbook(path)
writer = pd.ExcelWriter(path, engine = "openpyxl") 
writer.book = book

# create data in .xlsx format for input to the .vol format converter
for i in dfz["SEGMMENT"].unique()[1:]:
    for j in dfz["K"].unique()[1:8]:
        if i == "NW" and j == "BKX":
            pass
        else: 
            df = pd.read_excel("D:\GG\MINAS\Pressure Sumur Minas RFT.xlsx", sheet_name = "Pressure HM Tiga")
            df = df.groupby(["SEGMMENT","K","UWI","MISC_TEST_DATE_1"]).max().reset_index()
            df = df[((df["SEGMMENT"]==i)|(df["SEGMMENT"]=="ALL")) & ((df["K"]==j)|(df["K"]=="All"))]
            visualize_sand_segment(df)
            dfk = pd.DataFrame()
            dfk['Name'] = df['UWI']
            dfk['Day'] = df['MISC_TEST_DATE_1'].dt.day
            dfk['Month'] = df['MISC_TEST_DATE_1'].dt.month
            dfk['Year'] = df['MISC_TEST_DATE_1'].dt.year
            dfk['Pressure'] = df['Datum Pressure']
            dfk.loc[dfk['Name'].str.contains("MINA", case=False), "Name"] = str(j)[2:] + "_" + str(i) 
            dfk.loc[len(dfk.index)] = ["1","","","",""]
            dfk.to_excel(writer, sheet_name = str(j)[2:] + "_" + str(i), index = False)
 
writer.save()
writer.close()

# convert excel to .vol format
start = timeit.default_timer()

wb = xlsxwriter.Workbook(r"D:\GG\Pressure by Sand and by Segment.xlsx")

for segment in dfz["SEGMMENT"].unique()[1:]:
    for sand in dfz["K"].unique()[1:8]:
        if segment == "NW" and sand == "BKX":
            pass
        else: 
            sheet1 = wb.add_worksheet(str(sand)[2:] + "_" + str(segment))

            row_count = 3
            df_pressure = pd.read_excel(path, sheet_name = str(sand)[2:] + "_" + str(segment))

            sheet1.write(0, 0, '*FIELD')
            sheet1.write(1, 0, '*DAY')
            sheet1.write(1, 1, '*MONTH')
            sheet1.write(1, 2, '*YEAR')
            sheet1.write(1, 3, '*PRESS')
            sheet1.write(2, 0, '*NAME')

            for i in df_pressure.index :
                sheet1.write(row_count, 0, int(df_pressure['Day'][i]))
                sheet1.write(row_count, 1, int(df_pressure['Month'][i]))
                sheet1.write(row_count, 2, int(df_pressure['Year'][i]))
                sheet1.write(row_count, 3, int(df_pressure['Pressure'][i]))

                if i == 1 :
                    sheet1.write(2, 1, '"'+str(df_pressure['Name'][i])+'"')
                
                if str(df_pressure['Name'][i+1]) == '1' : 
                    break

                if str(df_pressure['Name'][i]) != str(df_pressure['Name'][i+1]) :
                    row_count+=2
                    sheet1.write(row_count, 0, '*NAME')
                    sheet1.write(row_count, 1, '"'+str(df_pressure['Name'][i+1])+'"')
                    row_count+=1

                elif str(df_pressure['Name'][i]) == str(df_pressure['Name'][i+1]) :
                    row_count+=1

wb.close()

stop = timeit.default_timer()
print('Time: ', stop - start)

# Find Average Pressure from RFT data and Convert It to .VOL format (by Sand)

In [ ]:
# Processing RFT data by sand and by segment

from openpyxl import load_workbook
import pandas as pd
import xlsxwriter
import timeit
import warnings
warnings.filterwarnings("ignore")

# read the data
dfz = pd.read_excel("D:\GG\MINAS\PRESSURE PBU dan SBHP MFFM 2022.xlsx", sheet_name = "PBU Selected")
path = "D:\GG\MINAS\PBU by Sand.xlsx"
book = load_workbook(path)
writer = pd.ExcelWriter(path, engine = "openpyxl") 
writer.book = book

# create data in .xlsx format for input to the .vol format converter
for i in dfz["SURVEY_SAND"].unique():
    df = pd.read_excel("D:\GG\MINAS\PRESSURE PBU dan SBHP MFFM 2022.xlsx", sheet_name = "PBU Selected")
    df = df.groupby(["SURVEY_SAND","UWI","TEST_DATE"]).mean().reset_index()
    df = df.dropna()
    df = df.sort_values(by = "TEST_DATE")
    df = df[(df["SURVEY_SAND"] == i)]
    visualize_sand_segment(df)
    dfk = pd.DataFrame()
    dfk['Name'] = df['UWI'] 
    dfk['Day'] = df['TEST_DATE'].dt.day
    dfk['Month'] = df['TEST_DATE'].dt.month
    dfk['Year'] = df['TEST_DATE'].dt.year
    dfk['Pressure'] = df['DATUM_PRESS']
    dfk.loc[dfk['Name'].str.contains("MINA", case = False), "Name"] = str(i)
    dfk.loc[len(dfk.index)] = ["1","","","",""]
    dfk.to_excel(writer, sheet_name = str(i), index = False)

writer.save()
writer.close()

# convert excel to .vol format
start = timeit.default_timer()

wb = xlsxwriter.Workbook(r"D:\GG\Minas\PBU by Sand (VOL).xlsx")

for sand in dfz["SURVEY_SAND"].unique():
    sheet1 = wb.add_worksheet(str(sand))

    row_count = 3
    df_pressure = pd.read_excel(path, sheet_name = str(sand))

    sheet1.write(0, 0, '*FIELD')
    sheet1.write(1, 0, '*DAY')
    sheet1.write(1, 1, '*MONTH')
    sheet1.write(1, 2, '*YEAR')
    sheet1.write(1, 3, '*PRESS')
    sheet1.write(2, 0, '*NAME')

    for i in df_pressure.index :
        sheet1.write(row_count, 0, int(df_pressure['Day'][i]))
        sheet1.write(row_count, 1, int(df_pressure['Month'][i]))
        sheet1.write(row_count, 2, int(df_pressure['Year'][i]))
        sheet1.write(row_count, 3, int(df_pressure['Pressure'][i]))

        if i == 1 :
            sheet1.write(2, 1, '"'+str(df_pressure['Name'][i])+'"')
        
        if str(df_pressure['Name'][i+1]) == '1' : 
            break

        if str(df_pressure['Name'][i]) != str(df_pressure['Name'][i+1]) :
            row_count+=2
            sheet1.write(row_count, 0, '*NAME')
            sheet1.write(row_count, 1, '"'+str(df_pressure['Name'][i+1])+'"')
            row_count+=1

        elif str(df_pressure['Name'][i]) == str(df_pressure['Name'][i+1]) :
            row_count+=1

wb.close()

stop = timeit.default_timer()
print('Time: ', stop - start)

# Find Average Pressure from PBU/SBHP data and Convert It to .VOL format (by Segment by Sand)

In [ ]:
# Processing RFT data by sand and by segment

from openpyxl import load_workbook
import pandas as pd
import xlsxwriter
import timeit
import warnings
warnings.filterwarnings("ignore")

# read the data
dfz = pd.read_excel("D:\GG\MINAS\Pressure\Pressure PBU and SHBP\PRESSURE PBU dan SBHP MFFM 2022 2.xlsx", sheet_name = "SBHP Selected")
path = r"D:\GG\MINAS\Pressure\Pressure PBU and SHBP\SBHP segment sand.xlsx"
book = load_workbook(path)
writer = pd.ExcelWriter(path, engine = "openpyxl") 
writer.book = book

# create data in .xlsx format for input to the .vol format converter
for x in dfz["SEGMENT"].unique():
    for i in dfz["SURVEY_SAND"].unique():
        if x == "NW" and i == "BK":
            pass
        else:
            df = pd.read_excel("D:\GG\MINAS\Pressure\Pressure PBU and SHBP\PRESSURE PBU dan SBHP MFFM 2022 2.xlsx", sheet_name = "SBHP Selected")
            df = df.groupby(["SEGMENT","SURVEY_SAND","UWI","TEST_DATE"]).mean().reset_index()
            df = df.dropna()
            df = df.sort_values(by = "TEST_DATE")
            df = df[(df["SURVEY_SAND"] == i) & (df["SEGMENT"] == x)]
            visualize_sand_segment2(df)
            dfk = pd.DataFrame()
            dfk['Name'] = df['UWI'] 
            dfk['Day'] = df['TEST_DATE'].dt.day
            dfk['Month'] = df['TEST_DATE'].dt.month
            dfk['Year'] = df['TEST_DATE'].dt.year
            dfk['Pressure'] = df['DATUM_PRESS']
            dfk.loc[dfk['Name'].str.contains("MINA", case = False), "Name"] = str(i)
            dfk.loc[len(dfk.index)] = ["1","","","",""]
            dfk.to_excel(writer, sheet_name = str(i) + "_" + str(x) , index = False)

writer.save()
writer.close()

# convert excel to .vol format
start = timeit.default_timer()

wb = xlsxwriter.Workbook(r"D:\GG\Minas\SBHP by Sand by Segment (VOL).xlsx")

for segment in dfz["SEGMENT"].unique():
    for sand in dfz["SURVEY_SAND"].unique():
        if segment == "NW" and sand == "BK":
            pass
        else:
            sheet1 = wb.add_worksheet(str(sand) + "_" + str(segment))

            row_count = 3
            df_pressure = pd.read_excel(path, sheet_name = str(sand) + "_" + str(segment))

            sheet1.write(0, 0, '*FIELD')
            sheet1.write(1, 0, '*DAY')
            sheet1.write(1, 1, '*MONTH')
            sheet1.write(1, 2, '*YEAR')
            sheet1.write(1, 3, '*PRESS')
            sheet1.write(2, 0, '*NAME')

            for i in df_pressure.index :
                sheet1.write(row_count, 0, int(df_pressure['Day'][i]))
                sheet1.write(row_count, 1, int(df_pressure['Month'][i]))
                sheet1.write(row_count, 2, int(df_pressure['Year'][i]))
                sheet1.write(row_count, 3, int(df_pressure['Pressure'][i]))

                if i == 1 :
                    sheet1.write(2, 1, '"'+str(df_pressure['Name'][i])+'"')
                

                if str(df_pressure['Name'][i+1]) == '1' : 
                    break

                if str(df_pressure['Name'][i]) != str(df_pressure['Name'][i+1]) :
                    row_count+=2
                    sheet1.write(row_count, 0, '*NAME')
                    sheet1.write(row_count, 1, '"'+str(df_pressure['Name'][i+1])+'"')
                    row_count+=1

                elif str(df_pressure['Name'][i]) == str(df_pressure['Name'][i+1]) :
                    row_count+=1

wb.close()

stop = timeit.default_timer()
print('Time: ', stop - start)

# Find Average Pressure from PBU/SBHP data and Convert It to .VOL format (FIELD)

In [ ]:
# Processing RFT data by sand and by segment

from openpyxl import load_workbook
import pandas as pd
import xlsxwriter
import timeit
import warnings
warnings.filterwarnings("ignore")

# read the data
dfz = pd.read_excel("D:\GG\MINAS\Pressure\Pressure PBU and SHBP\PRESSURE PBU dan SBHP MFFM 2022 2.xlsx", sheet_name = "SBHP Selected")
path = r"D:\GG\MINAS\Pressure\Pressure PBU and SHBP\SBHP FIELD.xlsx"
book = load_workbook(path)
writer = pd.ExcelWriter(path, engine = "openpyxl") 
writer.book = book

# create data in .xlsx format for input to the .vol format converter
df = pd.read_excel("D:\GG\MINAS\Pressure\Pressure PBU and SHBP\PRESSURE PBU dan SBHP MFFM 2022 2.xlsx", sheet_name = "SBHP Selected")
df = df.groupby(["UWI","TEST_DATE"]).mean().reset_index()

# visualize_sand_segment2(df)
dfk = pd.DataFrame()
dfk['Name'] = df['UWI'] 
dfk['Day'] = df['TEST_DATE'].dt.day
dfk['Month'] = df['TEST_DATE'].dt.month
dfk['Year'] = df['TEST_DATE'].dt.year
dfk['Pressure'] = df['DATUM_PRESS']
dfk.loc[dfk['Name'].str.contains("MINA", case = False), "Name"] = "FIELD"
dfk.loc[len(dfk.index)] = ["1","","","",""]
dfk.to_excel(writer, sheet_name = "FIELD", index = False)

writer.save()
writer.close()

# convert excel to .vol format
start = timeit.default_timer()

wb = xlsxwriter.Workbook(r"D:\GG\Minas\SBHP (VOL).xlsx")
sheet1 = wb.add_worksheet("FIELD")

row_count = 3
df_pressure = pd.read_excel(path, sheet_name = "FIELD")

sheet1.write(0, 0, '*FIELD')
sheet1.write(1, 0, '*DAY')
sheet1.write(1, 1, '*MONTH')
sheet1.write(1, 2, '*YEAR')
sheet1.write(1, 3, '*PRESS')
sheet1.write(2, 0, '*NAME')

for i in df_pressure.index :
    sheet1.write(row_count, 0, int(df_pressure['Day'][i]))
    sheet1.write(row_count, 1, int(df_pressure['Month'][i]))
    sheet1.write(row_count, 2, int(df_pressure['Year'][i]))
    sheet1.write(row_count, 3, int(df_pressure['Pressure'][i]))

    if i == 1 :
        sheet1.write(2, 1, '"'+str(df_pressure['Name'][i])+'"')
    

    if str(df_pressure['Name'][i+1]) == '1': 
        break

    if str(df_pressure['Name'][i]) != str(df_pressure['Name'][i+1]) :
        row_count+=2
        sheet1.write(row_count, 0, '*NAME')
        sheet1.write(row_count, 1, '"'+str(df_pressure['Name'][i+1])+'"')
        row_count+=1

    elif str(df_pressure['Name'][i]) == str(df_pressure['Name'][i+1]) :
        row_count+=1

wb.close()

stop = timeit.default_timer()
print('Time: ', stop - start)

# Create Perforation Format for Splitting Input


In [ ]:
import pandas as pd
import numpy as np

df = pd.read_excel("D:\GG\MINAS\Splitting\Komplesi Ready.xlsx", sheet_name="Sheet2")
df['UNITS FIELD'] = df['UNITS FIELD'].str.replace("\"", "")

i = 0
x = 0
df['Well'] = np.nan

while i <= 198878:
    
    if "WELLNAME" in str(df['UNITS FIELD'][i]) and "WELLNAME" in str(df['UNITS FIELD'][i+1]):
    
        df = df.drop(index = i, axis=0)

        i += 1
        x = i

    else:

        while x <= 198878: 
            
            if "perforation" in str(df['UNITS FIELD'][x+1]):
                df['Well'].loc[x+1] = df['UNITS FIELD'][i][9:]
                x += 1
                print(df['UNITS FIELD'][i][9:])
            elif "squeeze" in str(df['UNITS FIELD'][x+1]):
                df['Well'].loc[x+1] = df['UNITS FIELD'][i][9:]
                x += 1
                print(df['UNITS FIELD'][i][9:])
            elif "plug" in str(df['UNITS FIELD'][x+1]):
                df['Well'].loc[x+1] = df['UNITS FIELD'][i][9:]
                x += 1
                print(df['UNITS FIELD'][i][9:])
            else:
                break 
            
        df = df.drop(index = i, axis=0)

        x = x + 1
        i = x
        print(i,x) 
    
    if i == 198878:
        break

df.reset_index()

In [ ]:
df.to_excel("D:\GG\MINAS\Perfo2.xlsx", index = False)

# Writing User Edit for Pseudopressure RFT Data for FM in Petrel 

In [ ]:
import pandas as pd
import datetime as dt
import xlsxwriter as xw

# read and tranform list of RFT data 
df = pd.read_excel("D:\GG\MINAS\Pressure\MFFM RFT Pressure Final.xlsx", sheet_name = "Pressure HM")
df.sort_values(by = "TEST_DATE").reset_index(drop=True)
df = df.groupby(["TEST_DATE", "GRID_NAME"]).mean().reset_index()
df['TEST_DATE'] = df['TEST_DATE'].dt.strftime('%d-%b-%Y')

# write the user edit format within Excel
wb = xw.Workbook(r'D:\GG\MINAS\Pressure\User Edit RFT.xlsx')
sheet = wb.add_worksheet('User Edit RFT')

sheet.write(0, 0, "######################################")
sheet.write(1, 0, "# IXFVERSION: 2019.3 Petrel") 
sheet.write(2, 0, "######################################")
sheet.write(4, 0, "START")

rownum = 7

for i in df['TEST_DATE'].unique():
    
    sheet.write(rownum, 0, "DATE")
    sheet.write(rownum, 1, "\"" + str(i) + "\"")
    sheet.write(rownum + 2, 0, "RFTPLTReport")
    sheet.write(rownum + 2, 1, "\"RFTPLTReport\"")
    sheet.write(rownum + 2, 2, "[") 
    sheet.write(rownum + 3, 1, "WellNames")
    sheet.write(rownum + 3, 2, "WellRFTFlags")
    sheet.write(rownum + 3, 3, "WellPLTFlags")
    sheet.write(rownum + 3, 4, "WellSegmentFlags")

    if len(df[df['TEST_DATE'] == i]) >= 1:
        
        j = 1
        while j <= len(df[df['TEST_DATE'] == i]):

            sheet.write(rownum + 3 + j, 1,  "\"" + str(df[df['TEST_DATE'] == i]["GRID_NAME"].unique()[j-1]) + "\"")
            sheet.write(rownum + 3 + j, 2, "\"True\"")
            sheet.write(rownum + 3 + j, 3, "\"False\"")
            sheet.write(rownum + 3 + j, 4, "\"False\"")
 
            j += 1
    
    sheet.write(rownum + 3 + j, 0, "]")
    sheet.write(rownum + 5 + j, 0, "RFTPLTReport")
    sheet.write(rownum + 5 + j, 1, "\"" + "RFTPLTReport" + "\"")
    sheet.write(rownum + 5 + j, 2, "{")
    sheet.write(rownum + 6 + j, 0, "Frequency=ONE_TARGET_TIME")
    sheet.write(rownum + 7 + j, 0, "Period=1")
    sheet.write(rownum + 8 + j, 0, "On=\"True\"")
    sheet.write(rownum + 9 + j, 0, "FileFormat=BINARY")
    sheet.write(rownum + 10 + j, 0, "Units=ECLIPSE_FIELD")
    sheet.write(rownum + 11 + j, 0, "}")

    rownum = rownum + 13 + j

wb.close()

# Converter from xlsx to .VOL format

In [ ]:
from openpyxl import load_workbook
import pandas as pd
import xlsxwriter
import timeit
import warnings
warnings.filterwarnings("ignore")

# read the data
start = timeit.default_timer()

df = pd.read_excel("D:\GG\MINAS\Pressure\MFFM RFT Pressure Final.xlsx", sheet_name = "Pressure HM")
df = df.sort_values(by = "TEST_DATE")
dfk = pd.DataFrame()
dfk['Name'] = df['GRID_NAME'] 
dfk['Day'] = df['TEST_DATE'].dt.day
dfk['Month'] = df['TEST_DATE'].dt.month
dfk['Year'] = df['TEST_DATE'].dt.year
dfk['Pressure'] = df['DATUM_PRESSURE']
dfk.loc[len(dfk.index)] = ["1","","","",""]
dfk.to_excel("D:\GG\MINAS\Pressure\Pressure HM Raw.xlsx", sheet_name = "FIELD", index = False)

wb = xlsxwriter.Workbook(r"D:\GG\Minas\Raw (Vol).xlsx")
sheet1 = wb.add_worksheet("FIELD")

row_count = 3
df_pressure = pd.read_excel("D:\GG\MINAS\Pressure\Pressure HM Raw.xlsx", sheet_name = "FIELD")

sheet1.write(0, 0, '*FIELD')
sheet1.write(1, 0, '*DAY')
sheet1.write(1, 1, '*MONTH')
sheet1.write(1, 2, '*YEAR')
sheet1.write(1, 3, '*PRESS')
sheet1.write(2, 0, '*NAME')

for i in df_pressure.index :
    sheet1.write(row_count, 0, int(df_pressure['Day'][i]))
    sheet1.write(row_count, 1, int(df_pressure['Month'][i]))
    sheet1.write(row_count, 2, int(df_pressure['Year'][i]))
    sheet1.write(row_count, 3, int(df_pressure['Pressure'][i]))

    if i == 1 :
        sheet1.write(2, 1, '"'+str(df_pressure['Name'][i])+'"')
    

    if str(df_pressure['Name'][i+1]) == '1': 
        break

    if str(df_pressure['Name'][i]) != str(df_pressure['Name'][i+1]) :
        row_count+=2
        sheet1.write(row_count, 0, '*NAME')
        sheet1.write(row_count, 1, '"'+str(df_pressure['Name'][i+1])+'"')
        row_count+=1

    elif str(df_pressure['Name'][i]) == str(df_pressure['Name'][i+1]) :
        row_count+=1

wb.close()

stop = timeit.default_timer()
print('Time: ', stop - start)

# Create 45 Degree Plot with Hovering Anotations

### Cara 1

In [2]:
import plotly.express as px
import plotly.graph_objects as go
import pandas as pd

#read the data
df = pd.read_excel("D:\GG\MINAS\Crossplot per Segment Case 97_Rev.xlsx", sheet_name = "Crossplot 3")

for segment in df["Segment"].unique():
    # read the data
    df = pd.read_excel("D:\GG\MINAS\Crossplot per Segment Case 97_Rev.xlsx", sheet_name = "Crossplot 3")
    df = df[(df["Segment"] == segment)]
    fig = px.scatter(df, x = "Observed", y = "Simulation", hover_name = "Well", color = "Percentage", title = str(segment), 
    hover_data = ["Missmatch"], template = "plotly_dark")
    # fig2 = px.line(df, df["Observed"], df["Observed2"], template = "plotly_dark")
    # fig = go.Figure(data = fig1.data + fig2.data, template="plotly_dark")
    fig.show()

### Cara 2 (Per Segment)

In [1]:
import plotly.graph_objects as go
import pandas as pd

#read the data
df = pd.read_excel("D:\GG\MINAS\Crossplot per Segment Case 97_Rev.xlsx", sheet_name = "Crossplot 3")

for segment in df["Segment"].unique():
    
    # read the data
    df = pd.read_excel("D:\GG\MINAS\Crossplot per Segment Case 97_Rev.xlsx", sheet_name = "Crossplot 3")
    df = df[(df["Segment"] == segment)]
    
    fig = go.Figure()

    fig.add_trace(go.Scatter(x = df["Observed"], y = df["Simulation"], mode = 'markers', 
                    hovertext = df["Well"], marker = dict(
                        color = df["Percentage"],
                        colorscale = "Inferno",
                        showscale = True
                    ), showlegend = False))

    fig.add_trace(go.Scatter(x = df["Observed"], y = df["Observed2"], mode = 'lines', 
                    marker = dict(
                        color = "darkgrey"
                    ), showlegend = False))

    fig.update_layout(title_text = "Liquid Missmatch " + segment + " Segment")
    fig.update_xaxes(title = "Observed")
    fig.update_yaxes(title = "Simualtion")
    fig.layout.template = 'plotly_dark'

    fig.show()

### Cara 3 (Per Grid)

In [2]:
import plotly.graph_objects as go
import pandas as pd
import plotly.subplots as make_subplots

#read the data
df = pd.read_excel("D:\GG\MINAS\Crossplot per Segment Case 97_Rev.xlsx", sheet_name = "Crossplot 4")
df = df.sort_values(by = "Grid")

# fig = make_subplots(rows=11, cols=4)

for grid in df["Grid"].unique():
    
    # read the data
    df = pd.read_excel("D:\GG\MINAS\Crossplot per Segment Case 97_Rev.xlsx", sheet_name = "Crossplot 4")
    df = df[(df["Grid"] == grid)]
    
    fig = go.Figure()

    fig.add_trace(go.Scatter(x = df["Observed"], y = df["Simulation"], mode = 'markers+text', 
                    hovertext = df["Well"], marker = dict(
                        color = df["Percentage"],
                        colorscale = "Inferno",
                        showscale = True
                    ), showlegend = False,
                    text=df['Well'],
                    textposition="bottom center"))

    fig.add_trace(go.Scatter(x = df["Observed"], y = df["Observed2"], mode = 'lines', 
                    marker = dict(
                        color = "darkgrey"
                    ), showlegend = False))

    fig.update_layout(title_text = "Liquid Missmatch in " + grid + " (" + str(len(df)) + " Wells)", height=500, width=500)
    fig.update_xaxes(title = "Observed")
    fig.update_yaxes(title = "Simualtion")
    fig.layout.template = 'plotly_dark'

    fig.show()

### Cara 4 (With Subplots)

In [60]:
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import pandas as pd

#read the data
df = pd.read_excel("D:\GG\MINAS\Crossplot per Segment Case 97_Rev.xlsx", sheet_name = "Crossplot 4")
df = df.sort_values(by = "Grid")

fig = make_subplots(rows = 11, cols = 4, subplot_titles = (["Liquid Missmatch in " + grid for grid in df["Grid"].unique()]))

j = 1
x = 1

for grid in df["Grid"].unique():

    if j % 5 == 0:
        j = 1
        x += 1
    else:
        pass
    
    # read the data
    df = pd.read_excel("D:\GG\MINAS\Crossplot per Segment Case 97_Rev.xlsx", sheet_name = "Crossplot 4")
    df = df[(df["Grid"] == grid)]
    
    fig.add_trace(go.Scatter(x = df["Observed"], y = df["Simulation"], mode = 'markers',  hovertext = df["Well"], showlegend = False,
                    marker = dict(
                        color = df["Percentage"],
                        colorscale = "Inferno"
                    ),text = df['Well'], 
                    textposition = "bottom center"), 
                    row = x, col = j)

    fig.add_trace(go.Scatter(x = df["Observed"], y = df["Observed2"], mode = 'lines', showlegend = False),
                    row = x, col = j)

    fig.update_layout(title_text = "Liquid Missmatch (Simulation vs Observed)", height=5000, width=1850, colorscale_sequential = 'Inferno')
    fig.layout.template = 'plotly_dark'

    j += 1

fig.show()

# Restructuring Data Final Completion

In [ ]:
import pandas as pd

df = pd.read_excel("D:\GG\MINAS\Perfo2.xlsx", sheet_name = "Sheet1")
df = df.sort_values(by = ["Name", "Completion", "Date", "Top"]) 
df.to_excel("D:\GG\MINAS\Splitting\Final Completion.xlsx")